In [5]:
import cv2
import mediapipe as mp
import time
cap = cv2.VideoCapture(0)
mpHands = mp.solutions.hands.Hands() 
while True:
    success,img=cap.read()
    imgRGB=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    result=hands.process(imgRGB)
    cv2.imshow('HAND IMAGE',img)
    cv2.waitKey(1)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


KeyboardInterrupt: 

In [10]:
import cv2
import mediapipe as mp
import time

cap = cv2.VideoCapture(0)
mpHands = mp.solutions.hands.Hands()

while True:
    success, img = cap.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = mpHands.process(imgRGB)  # Corrected variable name
    results=mpHands.process(imgRGB)
    #print(results.multi_hand_landmark)
    if result.mult
    cv2.imshow('HAND IMAGE', img)
    cv2.waitKey(1)


AttributeError: type object 'SolutionOutputs' has no attribute 'multi_hand_landmark'

In [13]:
import cv2
import mediapipe as mp
import time

cap = cv2.VideoCapture(0)
mpHands = mp.solutions.hands.Hands()
mp.Draw=mp.solutions.drawing_utils

while True:
    success, img = cap.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = mpHands.process(imgRGB)  # Corrected variable name
    cv2.imshow('HAND IMAGE', img)
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            mp.Draw.draw_landmarks(img,handLms)
    cv2.waitKey(1)
    print(results.multi_hand_landmarks)  # Corrected attribute name


None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
[landmark {
  x: 0.68891275
  y: 1.1084081
  z: 2.9500202e-07
}
landmark {
  x: 0.63554096
  y: 1.0776739
  z: -0.020458486
}
landmark {
  x: 0.5871029
  y: 1.01404
  z: -0.04147425
}
landmark {
  x: 0.5416103
  y: 0.96210885
  z: -0.061595958
}
landmark {
  x: 0.5022872
  y: 0.9362762
  z: -0.08447728
}
landmark {
  x: 0.6177923
  y: 0.88178194
  z: -0.045517318
}
landmark {
  x: 0.5940935
  y: 0.7554241
  z: -0.071293905
}
landmark {
  x: 0.5830206
  y: 0.67411613
  z: -0.0

[landmark {
  x: 0.75196975
  y: 0.46376956
  z: 3.9543758e-07
}
landmark {
  x: 0.67792743
  y: 0.43239567
  z: -0.034127
}
landmark {
  x: 0.60603833
  y: 0.3489371
  z: -0.055202678
}
landmark {
  x: 0.54783654
  y: 0.2846559
  z: -0.07383688
}
landmark {
  x: 0.49548244
  y: 0.25278616
  z: -0.09245856
}
landmark {
  x: 0.63623387
  y: 0.14442754
  z: -0.027294429
}
landmark {
  x: 0.60285735
  y: 0.021353126
  z: -0.05454414
}
landmark {
  x: 0.5856652
  y: -0.053747505
  z: -0.07889499
}
landmark {
  x: 0.5746431
  y: -0.11686617
  z: -0.096387245
}
landmark {
  x: 0.6784595
  y: 0.111125365
  z: -0.029173706
}
landmark {
  x: 0.65130234
  y: -0.024242058
  z: -0.051857762
}
landmark {
  x: 0.63772917
  y: -0.0989615
  z: -0.07135442
}
landmark {
  x: 0.62850857
  y: -0.16108683
  z: -0.08531104
}
landmark {
  x: 0.72418785
  y: 0.10866085
  z: -0.03595993
}
landmark {
  x: 0.7062006
  y: -0.020417541
  z: -0.056465823
}
landmark {
  x: 0.69690585
  y: -0.09366605
  z: -0.0666776

[landmark {
  x: 0.7267995
  y: 0.63588476
  z: 5.541198e-07
}
landmark {
  x: 0.65552247
  y: 0.5693495
  z: -0.040145688
}
landmark {
  x: 0.6005771
  y: 0.46784925
  z: -0.06961197
}
landmark {
  x: 0.5558531
  y: 0.38374847
  z: -0.095834
}
landmark {
  x: 0.5066069
  y: 0.33287984
  z: -0.124216124
}
landmark {
  x: 0.6872232
  y: 0.279156
  z: -0.056352716
}
landmark {
  x: 0.67646974
  y: 0.12870498
  z: -0.09273259
}
landmark {
  x: 0.6756084
  y: 0.029972285
  z: -0.12063275
}
landmark {
  x: 0.67948645
  y: -0.05409819
  z: -0.14079753
}
landmark {
  x: 0.73890233
  y: 0.2735649
  z: -0.06206545
}
landmark {
  x: 0.75269383
  y: 0.10596757
  z: -0.095869064
}
landmark {
  x: 0.76475203
  y: -0.0004988909
  z: -0.120793626
}
landmark {
  x: 0.7773023
  y: -0.09412822
  z: -0.13856255
}
landmark {
  x: 0.78408116
  y: 0.30444273
  z: -0.07201123
}
landmark {
  x: 0.8044801
  y: 0.14470886
  z: -0.10527584
}
landmark {
  x: 0.81660926
  y: 0.038220063
  z: -0.124602675
}
landmar

[landmark {
  x: 0.7575818
  y: 0.857157
  z: 6.220373e-07
}
landmark {
  x: 0.6719913
  y: 0.83591014
  z: -0.04503727
}
landmark {
  x: 0.5948933
  y: 0.76819336
  z: -0.07821898
}
landmark {
  x: 0.5288285
  y: 0.71720606
  z: -0.10726384
}
landmark {
  x: 0.4659934
  y: 0.7000452
  z: -0.13872443
}
landmark {
  x: 0.644333
  y: 0.5488206
  z: -0.07017382
}
landmark {
  x: 0.5978578
  y: 0.3989836
  z: -0.111228116
}
landmark {
  x: 0.5704431
  y: 0.30041537
  z: -0.14106822
}
landmark {
  x: 0.5501015
  y: 0.20822415
  z: -0.16282268
}
landmark {
  x: 0.69665325
  y: 0.51061124
  z: -0.076990195
}
landmark {
  x: 0.6755517
  y: 0.32270747
  z: -0.11533701
}
landmark {
  x: 0.66074234
  y: 0.20029813
  z: -0.14269976
}
landmark {
  x: 0.6495542
  y: 0.0885053
  z: -0.16249259
}
landmark {
  x: 0.7519476
  y: 0.51259655
  z: -0.087778345
}
landmark {
  x: 0.73528075
  y: 0.3297795
  z: -0.1270482
}
landmark {
  x: 0.72175276
  y: 0.21080968
  z: -0.15000446
}
landmark {
  x: 0.710866

[landmark {
  x: 0.78736377
  y: 0.9110584
  z: 6.6743206e-07
}
landmark {
  x: 0.70184755
  y: 0.8843074
  z: -0.043636452
}
landmark {
  x: 0.62132645
  y: 0.8159203
  z: -0.074120894
}
landmark {
  x: 0.5523361
  y: 0.7659403
  z: -0.101496436
}
landmark {
  x: 0.48841023
  y: 0.7477169
  z: -0.13106
}
landmark {
  x: 0.68047076
  y: 0.5777134
  z: -0.049586687
}
landmark {
  x: 0.6403345
  y: 0.42704862
  z: -0.08141152
}
landmark {
  x: 0.620187
  y: 0.32734904
  z: -0.107560694
}
landmark {
  x: 0.6080896
  y: 0.23628682
  z: -0.1275087
}
landmark {
  x: 0.7369646
  y: 0.5381752
  z: -0.053488765
}
landmark {
  x: 0.7242202
  y: 0.34961155
  z: -0.08079686
}
landmark {
  x: 0.7176973
  y: 0.22833991
  z: -0.10432828
}
landmark {
  x: 0.71618974
  y: 0.12140575
  z: -0.122834586
}
landmark {
  x: 0.7926411
  y: 0.54238844
  z: -0.06252645
}
landmark {
  x: 0.78321093
  y: 0.36562866
  z: -0.09058803
}
landmark {
  x: 0.7755141
  y: 0.24817258
  z: -0.111164086
}
landmark {
  x: 0.

...

[landmark {
  x: 0.50264126
  y: 1.0255696
  z: 7.989696e-08
}
landmark {
  x: 0.5063957
  y: 0.9437246
  z: -0.012545289
}
landmark {
  x: 0.48841617
  y: 0.8780838
  z: -0.02230206
}
landmark {
  x: 0.4594795
  y: 0.8275597
  z: -0.028377768
}
landmark {
  x: 0.43372935
  y: 0.7922822
  z: -0.035739183
}
landmark {
  x: 0.51546866
  y: 0.9473802
  z: -0.041117337
}
landmark {
  x: 0.481278
  y: 0.8933986
  z: -0.057102755
}
landmark {
  x: 0.45306805
  y: 0.8614565
  z: -0.06630675
}
landmark {
  x: 0.43579078
  y: 0.8517929
  z: -0.070645876
}
landmark {
  x: 0.49466845
  y: 0.99949664
  z: -0.039557543
}
landmark {
  x: 0.4489939
  y: 0.94493866
  z: -0.052576
}
landmark {
  x: 0.42746395
  y: 0.912099
  z: -0.0563629
}
landmark {
  x: 0.4233839
  y: 0.9080989
  z: -0.05968855
}
landmark {
  x: 0.47040105
  y: 1.0392891
  z: -0.037000187
}
landmark {
  x: 0.4255982
  y: 0.99276423
  z: -0.046529062
}
landmark {
  x: 0.40576574
  y: 0.962627
  z: -0.045375682
}
landmark {
  x: 0.401

[landmark {
  x: 0.4992601
  y: 1.0675393
  z: -3.118971e-07
}
landmark {
  x: 0.5023938
  y: 0.97361535
  z: -0.0135029135
}
landmark {
  x: 0.49564168
  y: 0.8864206
  z: -0.03289167
}
landmark {
  x: 0.47651753
  y: 0.8192909
  z: -0.04992311
}
landmark {
  x: 0.46122524
  y: 0.75977933
  z: -0.067608014
}
landmark {
  x: 0.54896003
  y: 0.90346307
  z: -0.05917399
}
landmark {
  x: 0.49182045
  y: 0.82734585
  z: -0.07885952
}
landmark {
  x: 0.4567279
  y: 0.8203101
  z: -0.08171201
}
landmark {
  x: 0.4359343
  y: 0.82786065
  z: -0.08357756
}
landmark {
  x: 0.5404471
  y: 0.9571472
  z: -0.066186756
}
landmark {
  x: 0.47699952
  y: 0.86005974
  z: -0.085216306
}
landmark {
  x: 0.43842986
  y: 0.85894823
  z: -0.08187471
}
landmark {
  x: 0.42250583
  y: 0.8740302
  z: -0.08205389
}
landmark {
  x: 0.52009416
  y: 1.0096861
  z: -0.07228253
}
landmark {
  x: 0.45716318
  y: 0.91762525
  z: -0.09083022
}
landmark {
  x: 0.4288928
  y: 0.9164196
  z: -0.080245376
}
landmark {
  

[landmark {
  x: 0.62271875
  y: 0.79684573
  z: 2.3402643e-07
}
landmark {
  x: 0.5720487
  y: 0.7542634
  z: -0.017224038
}
landmark {
  x: 0.5376818
  y: 0.66960055
  z: -0.025452573
}
landmark {
  x: 0.5114496
  y: 0.60360336
  z: -0.033723336
}
landmark {
  x: 0.4822355
  y: 0.55965835
  z: -0.041873917
}
landmark {
  x: 0.5956037
  y: 0.5579464
  z: -0.00216798
}
landmark {
  x: 0.58629483
  y: 0.46333984
  z: -0.013923571
}
landmark {
  x: 0.58067864
  y: 0.40428895
  z: -0.027010268
}
landmark {
  x: 0.5767443
  y: 0.3539443
  z: -0.037327297
}
landmark {
  x: 0.6274172
  y: 0.54971725
  z: -0.0057631163
}
landmark {
  x: 0.62552047
  y: 0.4453072
  z: -0.015444856
}
landmark {
  x: 0.6236259
  y: 0.37715194
  z: -0.028004326
}
landmark {
  x: 0.6225145
  y: 0.32097977
  z: -0.037501916
}
landmark {
  x: 0.6576267
  y: 0.5623655
  z: -0.012896112
}
landmark {
  x: 0.65979266
  y: 0.46288127
  z: -0.02334537
}
landmark {
  x: 0.66031975
  y: 0.40181816
  z: -0.032074716
}
landma

[landmark {
  x: 0.6394414
  y: 0.8134457
  z: 2.9316345e-07
}
landmark {
  x: 0.5822002
  y: 0.76206553
  z: -0.017805953
}
landmark {
  x: 0.5372825
  y: 0.67036176
  z: -0.025579985
}
landmark {
  x: 0.5033212
  y: 0.59998846
  z: -0.03393301
}
landmark {
  x: 0.47043815
  y: 0.5544766
  z: -0.04217863
}
landmark {
  x: 0.5933357
  y: 0.53317475
  z: 0.0011353762
}
landmark {
  x: 0.57973164
  y: 0.42521143
  z: -0.010842847
}
landmark {
  x: 0.5744295
  y: 0.36044437
  z: -0.024060369
}
landmark {
  x: 0.57320064
  y: 0.30288887
  z: -0.035070255
}
landmark {
  x: 0.62807006
  y: 0.52217054
  z: -0.004328668
}
landmark {
  x: 0.6241138
  y: 0.40103525
  z: -0.013607169
}
landmark {
  x: 0.62375253
  y: 0.32521826
  z: -0.025502559
}
landmark {
  x: 0.6244497
  y: 0.2625574
  z: -0.035432864
}
landmark {
  x: 0.66132426
  y: 0.5345228
  z: -0.014325086
}
landmark {
  x: 0.66203505
  y: 0.41956884
  z: -0.025583336
}
landmark {
  x: 0.6621523
  y: 0.35122573
  z: -0.033963278
}
landm

KeyboardInterrupt: 

In [15]:
import cv2
import mediapipe as mp
import time

cap = cv2.VideoCapture(0)
mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

while True:
    success, img = cap.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    cv2.imshow('HAND IMAGE', img)

    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS,
                                  mpDraw.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=4),
                                  mpDraw.DrawingSpec(color=(0,255, 0), thickness=2, circle_radius=2),
                                 )
    cv2.waitKey(1)
    print(results.multi_hand_landmarks)


None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


[landmark {
  x: 0.08151987
  y: 1.1413704
  z: 6.080831e-07
}
landmark {
  x: 0.117635146
  y: 1.0661703
  z: -0.032508086
}
landmark {
  x: 0.1266607
  y: 0.9669434
  z: -0.05007629
}
landmark {
  x: 0.12886308
  y: 0.8817794
  z: -0.062201854
}
landmark {
  x: 0.14315724
  y: 0.8181053
  z: -0.075468265
}
landmark {
  x: 0.05399728
  y: 0.88428456
  z: -0.054978713
}
landmark {
  x: 0.039507795
  y: 0.75085706
  z: -0.077532485
}
landmark {
  x: 0.023365567
  y: 0.6671209
  z: -0.09178559
}
landmark {
  x: 0.008282568
  y: 0.60186636
  z: -0.10150565
}
landmark {
  x: 0.0076400563
  y: 0.9125179
  z: -0.04759452
}
landmark {
  x: -0.029635463
  y: 0.8085921
  z: -0.06599845
}
landmark {
  x: -0.048700254
  y: 0.7472236
  z: -0.07617515
}
landmark {
  x: -0.06374121
  y: 0.70160645
  z: -0.0832258
}
landmark {
  x: -0.023939777
  y: 0.9535068
  z: -0.04118349
}
landmark {
  x: -0.063199565
  y: 0.87826836
  z: -0.055548225
}
landmark {
  x: -0.08554278
  y: 0.83120114
  z: -0.0609448

[landmark {
  x: 0.3330397
  y: 0.95226294
  z: 5.178157e-07
}
landmark {
  x: 0.34608084
  y: 0.7971157
  z: -0.034120444
}
landmark {
  x: 0.32210714
  y: 0.6334486
  z: -0.04772866
}
landmark {
  x: 0.29659304
  y: 0.50471795
  z: -0.059652124
}
landmark {
  x: 0.29354608
  y: 0.4002443
  z: -0.07170638
}
landmark {
  x: 0.20297344
  y: 0.6132914
  z: -0.022435391
}
landmark {
  x: 0.14647144
  y: 0.5024696
  z: -0.046097867
}
landmark {
  x: 0.10329521
  y: 0.4371824
  z: -0.06825203
}
landmark {
  x: 0.06524396
  y: 0.38398442
  z: -0.08511114
}
landmark {
  x: 0.16506028
  y: 0.6813899
  z: -0.02373874
}
landmark {
  x: 0.08915025
  y: 0.5796277
  z: -0.043888867
}
landmark {
  x: 0.03959039
  y: 0.5174342
  z: -0.06457104
}
landmark {
  x: -0.0022082925
  y: 0.46547142
  z: -0.08035845
}
landmark {
  x: 0.14597215
  y: 0.76386887
  z: -0.029684238
}
landmark {
  x: 0.071806565
  y: 0.672314
  z: -0.050192434
}
landmark {
  x: 0.024472684
  y: 0.61450183
  z: -0.066687204
}
landm

[landmark {
  x: 0.44800028
  y: 0.9388486
  z: 6.95149e-07
}
landmark {
  x: 0.48188984
  y: 0.7978348
  z: -0.045742914
}
landmark {
  x: 0.4808515
  y: 0.62342775
  z: -0.06260618
}
landmark {
  x: 0.4805349
  y: 0.47652084
  z: -0.07582773
}
landmark {
  x: 0.49825984
  y: 0.36953962
  z: -0.08786717
}
landmark {
  x: 0.36526182
  y: 0.54089
  z: -0.025651623
}
landmark {
  x: 0.32849488
  y: 0.39740616
  z: -0.047907405
}
landmark {
  x: 0.29860616
  y: 0.31042334
  z: -0.069411404
}
landmark {
  x: 0.2709564
  y: 0.23577079
  z: -0.08599297
}
landmark {
  x: 0.31921285
  y: 0.58860165
  z: -0.02134944
}
landmark {
  x: 0.26638907
  y: 0.44839537
  z: -0.038627226
}
landmark {
  x: 0.22736774
  y: 0.35792574
  z: -0.056917403
}
landmark {
  x: 0.19431067
  y: 0.2815949
  z: -0.07153586
}
landmark {
  x: 0.2887668
  y: 0.6616883
  z: -0.023599323
}
landmark {
  x: 0.23669788
  y: 0.53899807
  z: -0.041864447
}
landmark {
  x: 0.20277408
  y: 0.45816845
  z: -0.05726665
}
landmark {

[landmark {
  x: 0.4546206
  y: 1.0964469
  z: 5.421993e-07
}
landmark {
  x: 0.48325792
  y: 0.9338759
  z: -0.033728816
}
landmark {
  x: 0.47199005
  y: 0.75312215
  z: -0.04453869
}
landmark {
  x: 0.46555206
  y: 0.61037165
  z: -0.054520994
}
landmark {
  x: 0.4887619
  y: 0.508239
  z: -0.064794414
}
landmark {
  x: 0.34695312
  y: 0.7103665
  z: -0.01454266
}
landmark {
  x: 0.2931648
  y: 0.5786519
  z: -0.030844972
}
landmark {
  x: 0.2546517
  y: 0.5018978
  z: -0.047499996
}
landmark {
  x: 0.22020508
  y: 0.44085237
  z: -0.061522197
}
landmark {
  x: 0.30556965
  y: 0.7741733
  z: -0.016694698
}
landmark {
  x: 0.23570077
  y: 0.6488166
  z: -0.030390663
}
landmark {
  x: 0.1882951
  y: 0.5725654
  z: -0.047251686
}
landmark {
  x: 0.14815871
  y: 0.5127958
  z: -0.060461584
}
landmark {
  x: 0.28235126
  y: 0.8561663
  z: -0.02419524
}
landmark {
  x: 0.21314049
  y: 0.74378073
  z: -0.04065137
}
landmark {
  x: 0.1711323
  y: 0.67790383
  z: -0.055474866
}
landmark {
  

...

[landmark {
  x: 0.40402862
  y: 0.98565745
  z: 7.146196e-07
}
landmark {
  x: 0.47681665
  y: 0.8905549
  z: -0.04593896
}
landmark {
  x: 0.52608824
  y: 0.74893844
  z: -0.07082359
}
landmark {
  x: 0.5624852
  y: 0.631383
  z: -0.09241876
}
landmark {
  x: 0.60534954
  y: 0.548698
  z: -0.11445961
}
landmark {
  x: 0.4380164
  y: 0.57564604
  z: -0.04893681
}
landmark {
  x: 0.43717286
  y: 0.4168393
  z: -0.08210329
}
landmark {
  x: 0.43181622
  y: 0.31455415
  z: -0.10775969
}
landmark {
  x: 0.424631
  y: 0.2237117
  z: -0.12697166
}
landmark {
  x: 0.38092923
  y: 0.5811232
  z: -0.05061658
}
landmark {
  x: 0.3656111
  y: 0.40301186
  z: -0.08123121
}
landmark {
  x: 0.35564023
  y: 0.28548747
  z: -0.10586169
}
landmark {
  x: 0.345091
  y: 0.18582752
  z: -0.12412049
}
landmark {
  x: 0.33205736
  y: 0.6269988
  z: -0.05751541
}
landmark {
  x: 0.30252606
  y: 0.4688875
  z: -0.08812948
}
landmark {
  x: 0.2871228
  y: 0.3601726
  z: -0.11000892
}
landmark {
  x: 0.2752920

[landmark {
  x: 0.38884875
  y: 1.0176269
  z: 6.9748097e-07
}
landmark {
  x: 0.4666826
  y: 0.9358419
  z: -0.04617106
}
landmark {
  x: 0.5220392
  y: 0.79796517
  z: -0.067561224
}
landmark {
  x: 0.5652753
  y: 0.67591745
  z: -0.08549998
}
landmark {
  x: 0.61311555
  y: 0.59876174
  z: -0.10289019
}
landmark {
  x: 0.4392288
  y: 0.6068251
  z: -0.03625252
}
landmark {
  x: 0.44416156
  y: 0.44856247
  z: -0.06357557
}
landmark {
  x: 0.44213426
  y: 0.34738478
  z: -0.08720807
}
landmark {
  x: 0.43913203
  y: 0.25605878
  z: -0.10524319
}
landmark {
  x: 0.38382643
  y: 0.6032876
  z: -0.034729693
}
landmark {
  x: 0.3780626
  y: 0.4279378
  z: -0.05837469
}
landmark {
  x: 0.37297347
  y: 0.31075424
  z: -0.0802303
}
landmark {
  x: 0.36811006
  y: 0.21017584
  z: -0.09705722
}
landmark {
  x: 0.33495694
  y: 0.63788086
  z: -0.039028224
}
landmark {
  x: 0.31922805
  y: 0.48045045
  z: -0.06232219
}
landmark {
  x: 0.31213573
  y: 0.37335598
  z: -0.08044347
}
landmark {
  

[landmark {
  x: 0.39553422
  y: 1.0610614
  z: 6.3219784e-07
}
landmark {
  x: 0.4746313
  y: 0.9861649
  z: -0.043899212
}
landmark {
  x: 0.52689165
  y: 0.85239506
  z: -0.062123314
}
landmark {
  x: 0.56102395
  y: 0.72058624
  z: -0.07667424
}
landmark {
  x: 0.6019286
  y: 0.6247318
  z: -0.09025406
}
landmark {
  x: 0.46341622
  y: 0.65627074
  z: -0.026716786
}
landmark {
  x: 0.47718439
  y: 0.5015539
  z: -0.050683115
}
landmark {
  x: 0.48139635
  y: 0.4015713
  z: -0.07207421
}
landmark {
  x: 0.483804
  y: 0.31401977
  z: -0.08808256
}
landmark {
  x: 0.4085707
  y: 0.6420643
  z: -0.023763822
}
landmark {
  x: 0.41310307
  y: 0.46485612
  z: -0.043655597
}
landmark {
  x: 0.41488028
  y: 0.3518825
  z: -0.0632632
}
landmark {
  x: 0.4162962
  y: 0.25443262
  z: -0.078111604
}
landmark {
  x: 0.35857382
  y: 0.66634345
  z: -0.0268998
}
landmark {
  x: 0.35425046
  y: 0.5045611
  z: -0.047362577
}
landmark {
  x: 0.35344675
  y: 0.39889032
  z: -0.06370947
}
landmark {
  

None
None
None
None
None
None
None
None
None
None
None


KeyboardInterrupt: 

In [17]:
import cv2
import mediapipe as mp

# Replace 'path/to/your/video/file.mp4' with the path to your pre-recorded video file
video_path = 'istockphoto-1008395954-640_adpp_is.mp4'

mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

cap = cv2.VideoCapture(video_path)

while True:
    success, img = cap.read()
    
    if not success:
        # End of video
        break
    
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    cv2.imshow('HAND IMAGE', img)

    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS,
                                  mpDraw.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=4),
                                  mpDraw.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                                 )
    
    if cv2.waitKey(1) & 0xFF == 27:  # Press 'Esc' key to exit
        break

cap.release()
cv2.destroyAllWindows()


In [18]:
import cv2
import mediapipe as mp

# Replace 'path/to/your/video/file.mp4' with the path to your pre-recorded video file
video_path = 'istockphoto-1008395954-640_adpp_is.mp4'

mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

cap = cv2.VideoCapture(video_path)

while True:
    success, img = cap.read()
    
    if not success:
        # End of video
        break
    
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    cv2.imshow('HAND IMAGE', img)

    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS,
                                  mpDraw.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=6),
                                  mpDraw.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
                                 )
    
    if cv2.waitKey(1) & 0xFF == 27:  # Press 'Esc' key to exit
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [20]:
import cv2
import mediapipe as mp

# Replace 'path/to/your/video/file.mp4' with the path to your pre-recorded video file
video_path = 'hand-video'

mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

cap = cv2.VideoCapture(video_path)

while True:
    success, img = cap.read()
    
    if not success:
        # End of video
        break
    
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    cv2.imshow('HAND IMAGE', img)

    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            # Draw palm points
            for id, lm in enumerate(handLms.landmark):
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                if id == 0:
                    cv2.circle(img, (cx, cy), 10, (255, 0, 0), -1)
                else:
                    cv2.circle(img, (cx, cy), 5, (0, 0, 255), -1)

            # Draw connections between points
            mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS,
                                  mpDraw.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                                  mpDraw.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
                                 )
    
    if cv2.waitKey(1) & 0xFF == 27:  # Press 'Esc' key to exit
        break

cap.release()
cv2.destroyAllWindows()


[aac @ 0x10df54b50] Input buffer exhausted before END element found


In [22]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
  while cap.isOpened():
    success, image = cap.read()

    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)

    image.flags.writeable = False
    results = hands.process(image)
    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    cv2.imshow('MediaPipe Hands', image)
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()

KeyboardInterrupt: 

In [23]:
import cv2
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()

        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)

        image.flags.writeable = False
        results = hands.process(image)
        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.multi_hand_landmarks:
            for handLms in results.multi_hand_landmarks:
                for id, lm in enumerate(handLms.landmark):
                    print(f"Landmark {id}: {lm}")

                mp_drawing.draw_landmarks(
                    image, handLms, mp_hands.HAND_CONNECTIONS)

        cv2.imshow('MediaPipe Hands', image)
        if cv2.waitKey(5) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()


Landmark 0: x: 0.8667618
y: 0.85553896
z: 3.4208e-07

Landmark 1: x: 0.7933616
y: 0.7972055
z: -0.027890572

Landmark 2: x: 0.7299263
y: 0.70127666
z: -0.04547017

Landmark 3: x: 0.68562603
y: 0.6217456
z: -0.06116035

Landmark 4: x: 0.64346755
y: 0.5815237
z: -0.07767012

Landmark 5: x: 0.79082716
y: 0.52464443
z: -0.032398023

Landmark 6: x: 0.7693676
y: 0.38204134
z: -0.05605789

Landmark 7: x: 0.7614948
y: 0.2971541
z: -0.07544347

Landmark 8: x: 0.7574277
y: 0.22261854
z: -0.09025458

Landmark 9: x: 0.8407073
y: 0.5027443
z: -0.03803583

Landmark 10: x: 0.8327794
y: 0.3347826
z: -0.056308102

Landmark 11: x: 0.8315896
y: 0.23872276
z: -0.07200519

Landmark 12: x: 0.8319302
y: 0.16001779
z: -0.08453696

Landmark 13: x: 0.8893061
y: 0.51449865
z: -0.047342025

Landmark 14: x: 0.88648176
y: 0.35291854
z: -0.067659326

Landmark 15: x: 0.8828062
y: 0.2599432
z: -0.08030739

Landmark 16: x: 0.8803417
y: 0.18317929
z: -0.09005494

Landmark 17: x: 0.9397272
y: 0.5531092
z: -0.05932615

La

Landmark 0: x: 0.77546614
y: 0.76855063
z: 3.2344988e-07

Landmark 1: x: 0.70847476
y: 0.7174519
z: -0.022086028

Landmark 2: x: 0.648561
y: 0.6284116
z: -0.035648793

Landmark 3: x: 0.6021154
y: 0.55652434
z: -0.049017884

Landmark 4: x: 0.55838174
y: 0.51240045
z: -0.061985802

Landmark 5: x: 0.69540083
y: 0.45055032
z: -0.00930936

Landmark 6: x: 0.6683525
y: 0.32901478
z: -0.025824169

Landmark 7: x: 0.6547831
y: 0.25830865
z: -0.042449888

Landmark 8: x: 0.64570177
y: 0.19215822
z: -0.056285188

Landmark 9: x: 0.7362604
y: 0.42857796
z: -0.016665261

Landmark 10: x: 0.71711737
y: 0.2879357
z: -0.02929871

Landmark 11: x: 0.7074092
y: 0.203551
z: -0.043580376

Landmark 12: x: 0.69855624
y: 0.13221446
z: -0.055917792

Landmark 13: x: 0.7788654
y: 0.4376568
z: -0.028615955

Landmark 14: x: 0.77117723
y: 0.30379438
z: -0.043718044

Landmark 15: x: 0.7654761
y: 0.22295615
z: -0.053683463

Landmark 16: x: 0.7572798
y: 0.15043348
z: -0.062584184

Landmark 17: x: 0.8226017
y: 0.47126222
z

Landmark 0: x: 0.8225745
y: 0.79397666
z: 3.797329e-07

Landmark 1: x: 0.7562969
y: 0.7407847
z: -0.026622143

Landmark 2: x: 0.6984034
y: 0.654814
z: -0.04414732

Landmark 3: x: 0.6539078
y: 0.5806831
z: -0.060229566

Landmark 4: x: 0.6115659
y: 0.5322482
z: -0.07569918

Landmark 5: x: 0.74990684
y: 0.48155454
z: -0.01954018

Landmark 6: x: 0.72896147
y: 0.35698247
z: -0.03948438

Landmark 7: x: 0.7193932
y: 0.28393066
z: -0.057536755

Landmark 8: x: 0.71416396
y: 0.21738559
z: -0.07163664

Landmark 9: x: 0.792977
y: 0.46393415
z: -0.0253556

Landmark 10: x: 0.78086984
y: 0.32035154
z: -0.041000783

Landmark 11: x: 0.7758777
y: 0.23384047
z: -0.056561857

Landmark 12: x: 0.77225053
y: 0.16080302
z: -0.06925948

Landmark 13: x: 0.83591527
y: 0.47963855
z: -0.03555051

Landmark 14: x: 0.83470595
y: 0.3422977
z: -0.053298518

Landmark 15: x: 0.83390105
y: 0.2609703
z: -0.06413985

Landmark 16: x: 0.8308705
y: 0.18971124
z: -0.07282655

Landmark 17: x: 0.87887555
y: 0.521513
z: -0.0485421

Landmark 0: x: 0.820128
y: 0.80099684
z: 4.3035394e-07

Landmark 1: x: 0.75565124
y: 0.7499566
z: -0.025800396

Landmark 2: x: 0.6988185
y: 0.65917
z: -0.04196163

Landmark 3: x: 0.6561312
y: 0.58506
z: -0.056391608

Landmark 4: x: 0.6156003
y: 0.54197776
z: -0.07084423

Landmark 5: x: 0.7508937
y: 0.48684722
z: -0.021274332

Landmark 6: x: 0.7333892
y: 0.3614965
z: -0.038702648

Landmark 7: x: 0.7249317
y: 0.28877774
z: -0.05501535

Landmark 8: x: 0.7205134
y: 0.22087464
z: -0.06855765

Landmark 9: x: 0.7963286
y: 0.4700714
z: -0.026589755

Landmark 10: x: 0.7874991
y: 0.3260349
z: -0.03995654

Landmark 11: x: 0.78356254
y: 0.23792595
z: -0.054808117

Landmark 12: x: 0.78109914
y: 0.16116565
z: -0.067717515

Landmark 13: x: 0.84005225
y: 0.4859857
z: -0.03570985

Landmark 14: x: 0.84017813
y: 0.3498075
z: -0.051311176

Landmark 15: x: 0.83965915
y: 0.26252127
z: -0.06359111

Landmark 16: x: 0.8374553
y: 0.18307468
z: -0.07427569

Landmark 17: x: 0.8823252
y: 0.52620786
z: -0.047423515

Landmark 0: x: 0.7439237
y: 0.7767171
z: 4.320613e-07

Landmark 1: x: 0.67762035
y: 0.7214758
z: -0.025704041

Landmark 2: x: 0.6170265
y: 0.63533115
z: -0.042521317

Landmark 3: x: 0.56743205
y: 0.5683134
z: -0.05800074

Landmark 4: x: 0.5227128
y: 0.52628744
z: -0.07343356

Landmark 5: x: 0.66584367
y: 0.46025014
z: -0.01999398

Landmark 6: x: 0.6427712
y: 0.335375
z: -0.038678836

Landmark 7: x: 0.63106966
y: 0.26168853
z: -0.05529449

Landmark 8: x: 0.6235381
y: 0.19491187
z: -0.06860295

Landmark 9: x: 0.7093312
y: 0.43914524
z: -0.025187278

Landmark 10: x: 0.69579154
y: 0.29396528
z: -0.0403789

Landmark 11: x: 0.68781275
y: 0.20716143
z: -0.05517359

Landmark 12: x: 0.6818197
y: 0.13278773
z: -0.067483686

Landmark 13: x: 0.753094
y: 0.4503157
z: -0.03460599

Landmark 14: x: 0.7487154
y: 0.31017774
z: -0.051678594

Landmark 15: x: 0.7453348
y: 0.22591561
z: -0.06444494

Landmark 16: x: 0.74076045
y: 0.15262437
z: -0.074602365

Landmark 17: x: 0.79652935
y: 0.4857251
z: -0.04667

...

Landmark 0: x: 0.79951733
y: 0.9020786
z: 5.0976445e-07

Landmark 1: x: 0.7327721
y: 0.8383656
z: -0.030688712

Landmark 2: x: 0.67403984
y: 0.7551004
z: -0.0548871

Landmark 3: x: 0.6195083
y: 0.7013744
z: -0.07782037

Landmark 4: x: 0.56740415
y: 0.67124766
z: -0.10172825

Landmark 5: x: 0.72210586
y: 0.5564778
z: -0.032066442

Landmark 6: x: 0.6969451
y: 0.42053613
z: -0.057800192

Landmark 7: x: 0.6843735
y: 0.33983976
z: -0.08011308

Landmark 8: x: 0.67738265
y: 0.26209658
z: -0.09783472

Landmark 9: x: 0.76769483
y: 0.5384554
z: -0.04003349

Landmark 10: x: 0.7524826
y: 0.37825066
z: -0.06285599

Landmark 11: x: 0.74546605
y: 0.27876082
z: -0.08402973

Landmark 12: x: 0.74055964
y: 0.1897276
z: -0.10088158

Landmark 13: x: 0.81464887
y: 0.557588
z: -0.052260317

Landmark 14: x: 0.8190714
y: 0.40122664
z: -0.07711608

Landmark 15: x: 0.8199946
y: 0.3039498
z: -0.0944271

Landmark 16: x: 0.81682444
y: 0.21484661
z: -0.10751793

Landmark 17: x: 0.860909
y: 0.603637
z: -0.06712661

L

Landmark 0: x: 0.8001249
y: 0.92375135
z: 4.957352e-07

Landmark 1: x: 0.7337834
y: 0.8635956
z: -0.026032206

Landmark 2: x: 0.67379606
y: 0.77512074
z: -0.04436409

Landmark 3: x: 0.62115544
y: 0.72156847
z: -0.061791964

Landmark 4: x: 0.57176685
y: 0.6893295
z: -0.079494104

Landmark 5: x: 0.7123059
y: 0.57155764
z: -0.019461561

Landmark 6: x: 0.6850362
y: 0.4327277
z: -0.038626827

Landmark 7: x: 0.67226255
y: 0.3534193
z: -0.056953296

Landmark 8: x: 0.66456634
y: 0.27910194
z: -0.07175373

Landmark 9: x: 0.75819474
y: 0.55056876
z: -0.027170576

Landmark 10: x: 0.73785645
y: 0.39010015
z: -0.042033073

Landmark 11: x: 0.7283886
y: 0.29405126
z: -0.059394114

Landmark 12: x: 0.7210656
y: 0.2113756
z: -0.07390508

Landmark 13: x: 0.8054423
y: 0.56438637
z: -0.03917747

Landmark 14: x: 0.7996077
y: 0.41057396
z: -0.056279693

Landmark 15: x: 0.7956225
y: 0.31443512
z: -0.071264654

Landmark 16: x: 0.7897369
y: 0.22646537
z: -0.08378188

Landmark 17: x: 0.85272694
y: 0.604455
z: -0

Landmark 0: x: 0.8175193
y: 0.96095395
z: 4.7088116e-07

Landmark 1: x: 0.7460043
y: 0.8997841
z: -0.026606873

Landmark 2: x: 0.68454486
y: 0.7980455
z: -0.04433826

Landmark 3: x: 0.6311574
y: 0.73099357
z: -0.061591364

Landmark 4: x: 0.5789467
y: 0.68927044
z: -0.078695

Landmark 5: x: 0.7281932
y: 0.59384704
z: -0.013645096

Landmark 6: x: 0.6990687
y: 0.45643145
z: -0.03314124

Landmark 7: x: 0.6843842
y: 0.37540627
z: -0.05290219

Landmark 8: x: 0.67540735
y: 0.29966426
z: -0.06891338

Landmark 9: x: 0.77461505
y: 0.5721687
z: -0.022058887

Landmark 10: x: 0.75394434
y: 0.4110033
z: -0.03676098

Landmark 11: x: 0.7432664
y: 0.3108001
z: -0.054125138

Landmark 12: x: 0.7338326
y: 0.22615153
z: -0.06861781

Landmark 13: x: 0.8226935
y: 0.58555394
z: -0.03560453

Landmark 14: x: 0.81609637
y: 0.4296279
z: -0.052627236

Landmark 15: x: 0.81105
y: 0.3353371
z: -0.065365

Landmark 16: x: 0.80286515
y: 0.24970359
z: -0.07627715

Landmark 17: x: 0.87049127
y: 0.6255287
z: -0.05236945

L

Landmark 0: x: 0.21939553
y: 0.9840964
z: -5.3272555e-07

Landmark 1: x: 0.2898025
y: 0.9612931
z: -0.024401976

Landmark 2: x: 0.35281232
y: 0.93125117
z: -0.052829906

Landmark 3: x: 0.39887244
y: 0.91440064
z: -0.0858221

Landmark 4: x: 0.44590014
y: 0.9039442
z: -0.12035548

Landmark 5: x: 0.32022455
y: 0.67885554
z: -0.034740224

Landmark 6: x: 0.35024428
y: 0.7866588
z: -0.08432491

Landmark 7: x: 0.3625118
y: 0.8759328
z: -0.12481563

Landmark 8: x: 0.37328303
y: 0.9401316
z: -0.14736697

Landmark 9: x: 0.26161155
y: 0.70327306
z: -0.047289953

Landmark 10: x: 0.30264392
y: 0.8359763
z: -0.0898169

Landmark 11: x: 0.32134318
y: 0.9195196
z: -0.117693156

Landmark 12: x: 0.33123323
y: 0.97771597
z: -0.13392906

Landmark 13: x: 0.20359798
y: 0.75271493
z: -0.06317774

Landmark 14: x: 0.23816739
y: 0.8591182
z: -0.103161715

Landmark 15: x: 0.26043537
y: 0.93012834
z: -0.11649483

Landmark 16: x: 0.2716454
y: 0.9824394
z: -0.11858261

Landmark 17: x: 0.15027064
y: 0.8183634
z: -0.0

KeyboardInterrupt: 

In [24]:
import cv2
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()

        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)

        image.flags.writeable = False
        results = hands.process(image)
        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.multi_hand_landmarks:
            for handLms in results.multi_hand_landmarks:
                for id, lm in enumerate(handLms.landmark):
                    
                    
                    print(f"Landmark {id}: {lm.x}, {lm.y}")

                mp_drawing.draw_landmarks(
                    image, handLms, mp_hands.HAND_CONNECTIONS)

        cv2.imshow('MediaPipe Hands', image)
        if cv2.waitKey(5) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()


Landmark 0: 0.08638782799243927, 1.0751230716705322
Landmark 1: 0.13107125461101532, 1.0314133167266846
Landmark 2: 0.16901957988739014, 0.9601153135299683
Landmark 3: 0.1970919966697693, 0.9004132747650146
Landmark 4: 0.21552228927612305, 0.8475587368011475
Landmark 5: 0.1365191638469696, 0.8736851215362549
Landmark 6: 0.1477564573287964, 0.7483689188957214
Landmark 7: 0.14638696610927582, 0.6700668334960938
Landmark 8: 0.1446753591299057, 0.6039035320281982
Landmark 9: 0.08916458487510681, 0.8677282929420471
Landmark 10: 0.07959234714508057, 0.7334980964660645
Landmark 11: 0.07241715490818024, 0.6486344337463379
Landmark 12: 0.06828765571117401, 0.5797160863876343
Landmark 13: 0.047808192670345306, 0.8891080021858215
Landmark 14: 0.029878728091716766, 0.7640403509140015
Landmark 15: 0.020225591957569122, 0.6788040995597839
Landmark 16: 0.01568704843521118, 0.6079567670822144
Landmark 17: 0.015529274940490723, 0.9291993379592896
Landmark 18: -0.012700557708740234, 0.8494411110877991
L

Landmark 0: 0.7884670495986938, 0.710996687412262
Landmark 1: 0.7329235076904297, 0.6662930846214294
Landmark 2: 0.6827853322029114, 0.5854138731956482
Landmark 3: 0.6456584930419922, 0.5167814493179321
Landmark 4: 0.6093119382858276, 0.47842490673065186
Landmark 5: 0.7215506434440613, 0.4453979730606079
Landmark 6: 0.7026057839393616, 0.34152790904045105
Landmark 7: 0.6927202939987183, 0.28136229515075684
Landmark 8: 0.6860357522964478, 0.22513678669929504
Landmark 9: 0.7576790452003479, 0.4298465847969055
Landmark 10: 0.745431125164032, 0.30843088030815125
Landmark 11: 0.7394530177116394, 0.23681530356407166
Landmark 12: 0.7346000075340271, 0.17440924048423767
Landmark 13: 0.7939049601554871, 0.43974536657333374
Landmark 14: 0.7877473831176758, 0.32611459493637085
Landmark 15: 0.7825836539268494, 0.2557682693004608
Landmark 16: 0.7767433524131775, 0.19210752844810486
Landmark 17: 0.8305560350418091, 0.4676511883735657
Landmark 18: 0.8379510641098022, 0.38175836205482483
Landmark 19: 

Landmark 0: 0.7841824293136597, 0.7237772345542908
Landmark 1: 0.7320399284362793, 0.6765930652618408
Landmark 2: 0.6879579424858093, 0.594765841960907
Landmark 3: 0.6583937406539917, 0.5232182741165161
Landmark 4: 0.6271817684173584, 0.4751397371292114
Landmark 5: 0.7273166179656982, 0.4505833387374878
Landmark 6: 0.7110304236412048, 0.34676674008369446
Landmark 7: 0.7031129002571106, 0.28643798828125
Landmark 8: 0.6994290351867676, 0.229291170835495
Landmark 9: 0.7651078701019287, 0.4369156062602997
Landmark 10: 0.7585570216178894, 0.3162311315536499
Landmark 11: 0.7557394504547119, 0.24170120060443878
Landmark 12: 0.7536593675613403, 0.17604869604110718
Landmark 13: 0.8004764914512634, 0.45175886154174805
Landmark 14: 0.8055721521377563, 0.33726584911346436
Landmark 15: 0.8078523874282837, 0.26561516523361206
Landmark 16: 0.8079487085342407, 0.20011559128761292
Landmark 17: 0.8334532380104065, 0.48657774925231934
Landmark 18: 0.8483118414878845, 0.40425699949264526
Landmark 19: 0.85

Landmark 0: 0.7753638029098511, 0.7285836338996887
Landmark 1: 0.7284992933273315, 0.6762065887451172
Landmark 2: 0.6955576539039612, 0.6040660738945007
Landmark 3: 0.682691216468811, 0.5402082204818726
Landmark 4: 0.6648398041725159, 0.4831784665584564
Landmark 5: 0.7167208194732666, 0.45803576707839966
Landmark 6: 0.7078076004981995, 0.35616710782051086
Landmark 7: 0.7044407725334167, 0.29665011167526245
Landmark 8: 0.7036253213882446, 0.23897284269332886
Landmark 9: 0.7530232071876526, 0.44908127188682556
Landmark 10: 0.7466931939125061, 0.33899569511413574
Landmark 11: 0.7439404726028442, 0.27219998836517334
Landmark 12: 0.7418393492698669, 0.2088259756565094
Landmark 13: 0.7861601114273071, 0.46499744057655334
Landmark 14: 0.7879803776741028, 0.3648556172847748
Landmark 15: 0.7891067862510681, 0.3021017014980316
Landmark 16: 0.7880057096481323, 0.2411135733127594
Landmark 17: 0.8186243772506714, 0.4993036687374115
Landmark 18: 0.8256101012229919, 0.42112818360328674
Landmark 19: 0

Landmark 0: 0.7758871912956238, 0.7279902100563049
Landmark 1: 0.7264114618301392, 0.6725818514823914
Landmark 2: 0.6937610507011414, 0.5771365761756897
Landmark 3: 0.6864176392555237, 0.490061491727829
Landmark 4: 0.6734387874603271, 0.42307960987091064
Landmark 5: 0.7125298380851746, 0.4682844877243042
Landmark 6: 0.7010719180107117, 0.3686126470565796
Landmark 7: 0.6976614594459534, 0.3107687830924988
Landmark 8: 0.6962283253669739, 0.2550085783004761
Landmark 9: 0.7433120608329773, 0.4601028561592102
Landmark 10: 0.7344917058944702, 0.3436715304851532
Landmark 11: 0.7303687334060669, 0.2724604904651642
Landmark 12: 0.7281623482704163, 0.2131432592868805
Landmark 13: 0.7740838527679443, 0.4755670130252838
Landmark 14: 0.7623794078826904, 0.3940507471561432
Landmark 15: 0.7378398776054382, 0.4228079617023468
Landmark 16: 0.7193715572357178, 0.4569738507270813
Landmark 17: 0.8044490814208984, 0.5080448389053345
Landmark 18: 0.7871811985969543, 0.4547101557254791
Landmark 19: 0.7609663

...

Landmark 0: 0.732311487197876, 0.7153005003929138
Landmark 1: 0.6764683127403259, 0.6548452973365784
Landmark 2: 0.6298401951789856, 0.562014102935791
Landmark 3: 0.5965876579284668, 0.48453816771507263
Landmark 4: 0.5643090009689331, 0.42968040704727173
Landmark 5: 0.6828539371490479, 0.4154212474822998
Landmark 6: 0.6709809899330139, 0.30722951889038086
Landmark 7: 0.6657708883285522, 0.24436317384243011
Landmark 8: 0.6634413599967957, 0.188667893409729
Landmark 9: 0.7205400466918945, 0.40704113245010376
Landmark 10: 0.7195703983306885, 0.28487759828567505
Landmark 11: 0.7202838659286499, 0.21162356436252594
Landmark 12: 0.7200181484222412, 0.15229877829551697
Landmark 13: 0.7568958401679993, 0.42500945925712585
Landmark 14: 0.7631251215934753, 0.3121560513973236
Landmark 15: 0.7654167413711548, 0.24427512288093567
Landmark 16: 0.7651289105415344, 0.1871339976787567
Landmark 17: 0.7918168902397156, 0.4634169936180115
Landmark 18: 0.8120874166488647, 0.38107436895370483
Landmark 19: 0

Landmark 0: 0.7444953918457031, 0.7205197811126709
Landmark 1: 0.6876307725906372, 0.6589618921279907
Landmark 2: 0.638918936252594, 0.561867892742157
Landmark 3: 0.6040621399879456, 0.4839254915714264
Landmark 4: 0.5698276162147522, 0.4323822557926178
Landmark 5: 0.6948501467704773, 0.39909860491752625
Landmark 6: 0.6843695044517517, 0.2839830815792084
Landmark 7: 0.6806597709655762, 0.215892493724823
Landmark 8: 0.6795740723609924, 0.15632423758506775
Landmark 9: 0.7351252436637878, 0.3926621377468109
Landmark 10: 0.7365320920944214, 0.2621946930885315
Landmark 11: 0.7389768958091736, 0.18275468051433563
Landmark 12: 0.7396966814994812, 0.12144932150840759
Landmark 13: 0.7740243673324585, 0.41463780403137207
Landmark 14: 0.783383846282959, 0.2988635301589966
Landmark 15: 0.786618709564209, 0.22863687574863434
Landmark 16: 0.7868631482124329, 0.17009982466697693
Landmark 17: 0.810809314250946, 0.4586115777492523
Landmark 18: 0.8331538438796997, 0.37747928500175476
Landmark 19: 0.84487

Landmark 0: 0.7610148787498474, 0.7486296892166138
Landmark 1: 0.7061519622802734, 0.6871132850646973
Landmark 2: 0.6586791276931763, 0.5873486995697021
Landmark 3: 0.6248831152915955, 0.5066150426864624
Landmark 4: 0.5923852324485779, 0.4565117657184601
Landmark 5: 0.7173534035682678, 0.43334364891052246
Landmark 6: 0.7067615389823914, 0.3212711811065674
Landmark 7: 0.7036073803901672, 0.25733354687690735
Landmark 8: 0.7029404640197754, 0.20185038447380066
Landmark 9: 0.7564296722412109, 0.428615927696228
Landmark 10: 0.7575621604919434, 0.30305302143096924
Landmark 11: 0.7600123286247253, 0.22853422164916992
Landmark 12: 0.7615668177604675, 0.17054912447929382
Landmark 13: 0.7935190200805664, 0.45027756690979004
Landmark 14: 0.8010488748550415, 0.33991509675979614
Landmark 15: 0.803543210029602, 0.2712559700012207
Landmark 16: 0.8044054508209229, 0.2142251878976822
Landmark 17: 0.8285402655601501, 0.49320700764656067
Landmark 18: 0.8483825325965881, 0.4158146381378174
Landmark 19: 0.

Landmark 0: 0.8135347366333008, 0.9762825965881348
Landmark 1: 0.7753879427909851, 0.9627055525779724
Landmark 2: 0.7346699833869934, 0.9125576019287109
Landmark 3: 0.6986649036407471, 0.8837294578552246
Landmark 4: 0.6708542108535767, 0.8541530966758728
Landmark 5: 0.8007084131240845, 0.74915611743927
Landmark 6: 0.7972842454910278, 0.6288797855377197
Landmark 7: 0.795792281627655, 0.5642745494842529
Landmark 8: 0.7963117957115173, 0.5052734613418579
Landmark 9: 0.8394502997398376, 0.742701530456543
Landmark 10: 0.855137825012207, 0.6128535866737366
Landmark 11: 0.8695772290229797, 0.5363750457763672
Landmark 12: 0.8840264678001404, 0.47045448422431946
Landmark 13: 0.8685442209243774, 0.7680588364601135
Landmark 14: 0.8899811506271362, 0.6462517380714417
Landmark 15: 0.9078932404518127, 0.5683221220970154
Landmark 16: 0.9249565601348877, 0.499943345785141
Landmark 17: 0.890468418598175, 0.8079723119735718
Landmark 18: 0.9122246503829956, 0.7129478454589844
Landmark 19: 0.9280158877372

KeyboardInterrupt: 

In [ ]:
import cv2
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()

        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)

        image.flags.writeable = False
        results = hands.process(image)
        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.multi_hand_landmarks:
            for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
                print(f"Hand {idx} landmarks:")
                for lm_idx, landmark in enumerate(hand_landmarks.landmark):
                    print(f"Landmark {lm_idx}: {landmark}")

                mp_drawing.draw_landmarks(
                    image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        cv2.imshow('MediaPipe Hands', image)
        if cv2.waitKey(5) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()


In [28]:
import cv2
import mediapipe as mp
import pyautogui
webcam =cv2.VideoCapture(0)
while True:
    _,image=webcam.read()
    cv2.imshow('hand volume control using python',image)
    key=cv2.waitKey()
    if key==27:
        break
webcam.release()
cv2.destroyAllWindows()

In [30]:
import cv2

# Open the webcam (use 0 for default webcam)
webcam = cv2.VideoCapture(0)
#to capture our hands 
my_hands=mp.solutions.hands.Hands()
drawing_units=mp.solutions.drawing_units

while True:
    # Read a frame from the webcam
    ret, frame = webcam.read()
    #we need to convert our image into rgb format
    rgb_image =cv2.cvtColor()

    # Show the frame in a window titled 'Webcam'
    cv2.imshow('Webcam',image)
    #we need to convert our image into rgb format
    rgb_image =cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    output=my_hands.process(rgb_image)
    #collecting all hands from the varible hands
    hands =outputs.multi_hands_landmarks
    if hands:
        for hands in hands:
            drawing_utils.draw_landmarks(images,hand)
    cv2.imshow('Webcam',image)
    # Check for the 'Esc' key (ASCII code 27) to exit the loop
    if cv2.waitKey(1) == 27:
        break

# Release the webcam and close the window
webcam.release()
cv2.destroyAllWindows()


AttributeError: module 'mediapipe.python.solutions' has no attribute 'drawing_units'

In [ ]:
import cv2
import mediapipe as mp

# Open the webcam (use 0 for the default webcam)
webcam = cv2.VideoCapture(0)

# To capture our hands
my_hands = mp.solutions.hands.Hands()
drawing_utils = mp.solutions.drawing_utils

while True:
    # Read a frame from the webcam
    ret, image = webcam.read()
    frame_height.frame_width,ret = image.shape()

    # We need to convert our image into RGB format
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the frame with Mediapipe Hands
    output = my_hands.process(rgb_image)

    # Collecting all hands from the variable output
    if output.multi_hand_landmarks:
        for hand_landmarks in output.multi_hand_landmarks:
            drawing_utils.draw_landmarks(image, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS)
            #for collecting the landmarks 
            landmarks =hand.landmark
            #collecting id
            for id,landmark in enumerate(landmarks):
                x=int(landmark.x*frame_width)
                y=int(landmark.y*frame_height)
                if id==8:
                    cv2.circle(img=image,center(x,y),radious=8,color=(0,255,255),thickness=3))
                if id==4:
                    cv2.circle(img=image,center=(x,y,radious=8,color=(0,0,255)thickness=3))

    # Show the frame in a window titled 'Webcam'
    cv2.imshow('Webcam', image)

    # Check for the 'Esc' key (ASCII code 27) to exit the loop
    if cv2.waitKey(1) == 27:
        break

# Release the webcam and close the window
webcam.release()
cv2.destroyAllWindows()


In [36]:
import cv2
import mediapipe as mp
x1 = y1 = x2 = y2=0
# Open the webcam (use 0 for the default webcam)
webcam = cv2.VideoCapture(0)

# To capture our hands
my_hands = mp.solutions.hands.Hands()
drawing_utils = mp.solutions.drawing_utils

while True:
    # Read a frame from the webcam
    ret, image = webcam.read()
    frame_height, frame_width, _ = image.shape

    # We need to convert our image into RGB format
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the frame with Mediapipe Hands
    output = my_hands.process(rgb_image)

    # Collecting all hands from the variable output
    if output.multi_hand_landmarks:
        for hand_landmarks in output.multi_hand_landmarks:
            drawing_utils.draw_landmarks(image, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS)
            # For collecting the landmarks
            landmarks = hand_landmarks.landmark
            # Collecting id
            for id, landmark in enumerate(landmarks):
                x = int(landmark.x * frame_width)
                y = int(landmark.y * frame_height)
                if id == 8:
                    cv2.circle(image, (x, y), radius=8, color=(0, 255, 255), thickness=3)
                    x1=x
                    y1=y
                if id == 4:
                    cv2.circle(image, (x, y), radius=8, color=(0, 0, 255), thickness=3)
                    x2=x
                    y2=y
                cv2.line(image,(x1,y2),(x2,y2),(0,255,0),5)

    # Show the frame in a window titled 'Webcam'
    cv2.imshow('Webcam', image)

    # Check for the 'Esc' key (ASCII code 27) to exit the loop
    if cv2.waitKey(1) == 27:
        break

# Release the webcam and close the window
webcam.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [39]:
import cv2
import mediapipe as mp

x1 = y1 = x2 = y2 = 0

# Open the webcam (use 0 for the default webcam)
webcam = cv2.VideoCapture(0)

# To capture our hands
my_hands = mp.solutions.hands.Hands()
drawing_utils = mp.solutions.drawing_utils

while True:
    # Read a frame from the webcam
    ret, image = webcam.read()
    frame_height, frame_width, _ = image.shape

    # We need to convert our image into RGB format
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the frame with Mediapipe Hands
    output = my_hands.process(rgb_image)

    # Collecting all hands from the variable output
    if output.multi_hand_landmarks:
        for hand_landmarks in output.multi_hand_landmarks:
            drawing_utils.draw_landmarks(image, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS)
            # For collecting the landmarks
            landmarks = hand_landmarks.landmark
            # Collecting id
            for id, landmark in enumerate(landmarks):
                x = int(landmark.x * frame_width)
                y = int(landmark.y * frame_height)
                if id == 4:
                    cv2.circle(image, (x, y), radius=8, color=(0, 255, 255), thickness=3)
                    x1 = x
                    y1 = y
                if id == 8:
                    cv2.circle(image, (x, y), radius=8, color=(0, 0, 255), thickness=3)
                    x2 = x
                    y2 = y
                    distance=((x2-x1)**2 +(y2-y1)**(0.5))//4
                
                cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 5)
                if dist > 50:
                    
                    pyautogui.press('volumeup')
                else:
                    pyautogui.press('volumedown')
                              
                              
                              

    # Show the frame in a window titled 'Webcam'
    cv2.imshow('Webcam', image)

    # Check for the 'Esc' key (ASCII code 27) to exit the loop
    if cv2.waitKey(1) == 27:
        break

# Release the webcam and close the window
webcam.release()
cv2.destroyAllWindows()


TypeError: unsupported operand type(s) for //: 'complex' and 'int'

In [42]:
import cv2
import mediapipe as mp
import pyautogui

x1 = y1 = x2 = y2 = 0

# Open the webcam (use 0 for the default webcam)
webcam = cv2.VideoCapture(0)

# To capture our hands
my_hands = mp.solutions.hands.Hands()
drawing_utils = mp.solutions.drawing_utils

while True:
    # Read a frame from the webcam
    ret, image = webcam.read()
    frame_height, frame_width, ret = image.shape

    # We need to convert our image into RGB format
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the frame with Mediapipe Hands
    output = my_hands.process(rgb_image)

    # Collecting all hands from the variable output
    if output.multi_hand_landmarks:
        for hand_landmarks in output.multi_hand_landmarks:
            drawing_utils.draw_landmarks(image, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS)
            # For collecting the landmarks
            landmarks = hand_landmarks.landmark
            # Collecting id
            for id, landmark in enumerate(landmarks):
                x = int(landmark.x * frame_width)
                y = int(landmark.y * frame_height)
                if id == 4:
                    cv2.circle(image, (x, y), radius=8, color=(0, 255, 255), thickness=3)
                    x1 = x
                    y1 = y
                if id == 8:
                    cv2.circle(image, (x, y), radius=8, color=(0, 0, 255), thickness=3)
                    x2 = x
                    y2 = y
                    distance = ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5
                    cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 5)

                    if distance > 50:
                        pyautogui.press('volumeup')
                    else:
                        pyautogui.press('volumedown')

    # Show the frame in a window titled 'Webcam'
    cv2.imshow('Webcam', image)

    # Check for the 'Esc' key (ASCII code 27) to exit the loop
    if cv2.waitKey(1) == 27:
        break

# Release the webcam and close the window
webcam.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [44]:
import cv2
import mediapipe as mp
import pyautogui

x1 = y1 = x2 = y2 = 0

# Open the webcam (use 0 for the default webcam)
webcam = cv2.VideoCapture(0)

# To capture our hands
my_hands = mp.solutions.hands.Hands()
drawing_utils = mp.solutions.drawing_utils

# Set the initial volume level
current_volume = 50

while True:
    # Read a frame from the webcam
    ret, image = webcam.read()
    frame_height, frame_width, ret = image.shape

    # We need to convert our image into RGB format
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the frame with Mediapipe Hands
    output = my_hands.process(rgb_image)

    # Collecting all hands from the variable output
    if output.multi_hand_landmarks:
        for hand_landmarks in output.multi_hand_landmarks:
            drawing_utils.draw_landmarks(image, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS)
            # For collecting the landmarks
            landmarks = hand_landmarks.landmark
            # Collecting id
            for id, landmark in enumerate(landmarks):
                x = int(landmark.x * frame_width)
                y = int(landmark.y * frame_height)
                if id == 4:
                    cv2.circle(image, (x, y), radius=8, color=(0, 255, 255), thickness=3)
                    x1 = x
                    y1 = y
                if id == 8:
                    cv2.circle(image, (x, y), radius=8, color=(0, 0, 255), thickness=3)
                    x2 = x
                    y2 = y
                    distance = ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5
                    cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 5)

                    # Adjust the volume based on hand distance
                    if distance > 50:
                        current_volume += 1
                        if current_volume > 100:
                            current_volume = 100
                    else:
                        current_volume -= 1
                        if current_volume < 0:
                            current_volume = 0
                    pyautogui.press('volumedown', presses=current_volume)  # Set the volume level

    # Show the current volume as a bar on the webcam image
    cv2.rectangle(image, (20, 50), (20 + current_volume, 70), (0, 255, 0), -1)

    # Show the frame in a window titled 'Webcam'
    cv2.imshow('Webcam', image)

    # Check for the 'Esc' key (ASCII code 27) to exit the loop
    if cv2.waitKey(1) == 27:
        break

# Release the webcam and close the window
webcam.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [50]:
import cv2
import mediapipe as mp
import pyautogui

x1 = y1 = x2 = y2 = 0

# Open the webcam (use 0 for the default webcam)
webcam = cv2.VideoCapture(0)

# To capture our hands
my_hands = mp.solutions.hands.Hands()
drawing_utils = mp.solutions.drawing_utils

# Set the initial volume level
current_volume = 10

while True:
    # Read a frame from the webcam
    ret, image = webcam.read()
    frame_height, frame_width, ret = image.shape

    # We need to convert our image into RGB format
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the frame with Mediapipe Hands
    output = my_hands.process(rgb_image)

    # Collecting all hands from the variable output
    if output.multi_hand_landmarks:
        for hand_landmarks in output.multi_hand_landmarks:
            drawing_utils.draw_landmarks(image, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS)
            # For collecting the landmarks
            landmarks = hand_landmarks.landmark
            # Collecting id
            for id, landmark in enumerate(landmarks):
                x = int(landmark.x * frame_width)
                y = int(landmark.y * frame_height)
                if id == 4:
                    cv2.circle(image, (x, y), radius=8, color=(0, 255, 255), thickness=3)
                    x1 = x
                    y1 = y
                if id == 8:
                    cv2.circle(image, (x, y), radius=8, color=(0, 0, 255), thickness=3)
                    x2 = x
                    y2 = y
                    distance = ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5
                    cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 5)

                    # Adjust the volume based on hand distance
                    if distance > 50:
                        current_volume += 1
                        if current_volume > 100:
                            current_volume = 100
                    else:
                        current_volume -= 1
                        if current_volume < 0:
                            current_volume = 0
                    pyautogui.press('volumedown', presses=current_volume)  # Set the volume level

    # Show the current volume as a bar on the webcam image
    cv2.rectangle(image, (20, 50), (20 + current_volume, 70), (0, 255, 0), -1)

    # Show the frame in a window titled 'Webcam'
    cv2.imshow('Webcam', image)

    # Check for the 'Esc' key (ASCII code 27) to exit the loop
    if cv2.waitKey(1) == 27:
        break

# Release the webcam and close the window
webcam.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [ ]:
import cv2
import mediapipe as mp
import pyautogui

x1 = y1 = x2 = y2 = 0

# Open the webcam (use 0 for the default webcam)
webcam = cv2.VideoCapture(0)

# To capture our hands
my_hands = mp.solutions.hands.Hands()
drawing_utils = mp.solutions.drawing_utils

# Set the initial volume level
current_volume = 10

while True:
    # Read a frame from the webcam
    ret, image = webcam.read()
    frame_height, frame_width, _ = image.shape

    # We need to convert our image into RGB format
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the frame with Mediapipe Hands
    output = my_hands.process(rgb_image)

    # Collecting all hands from the variable output
    if output.multi_hand_landmarks:
        for hand_landmarks in output.multi_hand_landmarks:
            drawing_utils.draw_landmarks(image, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS)
            # For collecting the landmarks
            landmarks = hand_landmarks.landmark
            # Collecting id
            for id, landmark in enumerate(landmarks):
                x = int(landmark.x * frame_width)
                y = int(landmark.y * frame_height)
                if id == 4:
                    cv2.circle(image, (x, y), radius=8, color=(0, 255, 255), thickness=3)
                    x1 = x
                    y1 = y
                if id == 8:
                    cv2.circle(image, (x, y), radius=8, color=(0, 0, 255), thickness=3)
                    x2 = x
                    y2 = y
                    distance = ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5
                    cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 5)

                    # Adjust the volume based on hand distance
                    if distance > 50:
                        current_volume += 1
                        if current_volume > 100:
                            current_volume = 100
                    else:
                        current_volume -= 1
                        if current_volume < 0:
                            current_volume = 0

    # Show the current volume as a bar on the webcam image
    cv2.rectangle(image, (20, 50), (20 + current_volume, 70), (0, 255, 0), -1)

    # Show the frame in a window titled 'Webcam'
    cv2.imshow('Webcam', image)

    # Check for the 'Esc' key (ASCII code 27) to exit the loop
    if cv2.waitKey(1) == 27:
        break

# Release the webcam and close the window
webcam.release()
cv2.destroyAllWindows()


In [1]:
import cv2
import mediapipe as mp
import pyautogui

x1 = y1 = x2 = y2 = 0

# Open the webcam (use 0 for the default webcam)
webcam = cv2.VideoCapture(0)

# To capture our hands
my_hands = mp.solutions.hands.Hands()
drawing_utils = mp.solutions.drawing_utils

# Set the initial volume level
current_volume = 10

while True:
    # Read a frame from the webcam
    ret, image = webcam.read()
    frame_height, frame_width, _ = image.shape

    # We need to convert our image into RGB format
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the frame with Mediapipe Hands
    output = my_hands.process(rgb_image)

    # Collecting all hands from the variable output
    if output.multi_hand_landmarks:
        for hand_landmarks in output.multi_hand_landmarks:
            drawing_utils.draw_landmarks(image, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS)
            # For collecting the landmarks
            landmarks = hand_landmarks.landmark
            # Collecting id
            for id, landmark in enumerate(landmarks):
                x = int(landmark.x * frame_width)
                y = int(landmark.y * frame_height)
                if id == 4:
                    cv2.circle(image, (x, y), radius=8, color=(0, 255, 255), thickness=3)
                    x1 = x
                    y1 = y
                if id == 8:
                    cv2.circle(image, (x, y), radius=8, color=(0, 0, 255), thickness=3)
                    x2 = x
                    y2 = y
                    distance = ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5
                    cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 5)

                    # Adjust the volume based on hand distance
                    if distance > 50:
                        current_volume += 1
                        if current_volume > 100:
                            current_volume = 100
                    else:
                        current_volume -= 1
                        if current_volume < 0:
                            current_volume = 0

    # Show the current volume as a vertical bar on the webcam image
    volume_bar_height = int((current_volume / 100) * frame_height)
    cv2.rectangle(image, (frame_width - 50, frame_height), (frame_width - 30, frame_height - volume_bar_height),
                  (0, 255, 0), -1)
    
    # Display volume level text
    cv2.putText(image, f"Volume: {current_volume}", (frame_width - 100, frame_height - 30), cv2.FONT_HERSHEY_SIMPLEX,
                0.5, (255, 255, 255), 1, cv2.LINE_AA)

    # Show the frame in a window titled 'Webcam'
    cv2.imshow('Webcam', image)

    # Check for the 'Esc' key (ASCII code 27) to exit the loop
    if cv2.waitKey(1) == 27:
        break

# Release the webcam and close the window
webcam.release()
cv2.destroyAllWindows()


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


KeyboardInterrupt: 

In [6]:
import cv2
import mediapipe as mp

x1 = y1 = x2 = y2 = 0

# Open the webcam (use 0 for the default webcam)
webcam = cv2.VideoCapture(0)

# To capture our hands
my_hands = mp.solutions.hands.Hands()
drawing_utils = mp.solutions.drawing_utils

# Set the initial volume level
current_volume = 10
alpha = 0.2  # Smoothing factor for the exponential moving average

while True:
    # Read a frame from the webcam
    ret, image = webcam.read()
    frame_height, frame_width, _ = image.shape

    # We need to convert our image into RGB format
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the frame with Mediapipe Hands
    output = my_hands.process(rgb_image)

    # Collecting all hands from the variable output
    if output.multi_hand_landmarks:
        for hand_landmarks in output.multi_hand_landmarks:
            drawing_utils.draw_landmarks(image, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS)
            # For collecting the landmarks
            landmarks = hand_landmarks.landmark
            # Collecting id
            for id, landmark in enumerate(landmarks):
                x = int(landmark.x * frame_width)
                y = int(landmark.y * frame_height)
                if id == 4:
                    cv2.circle(image, (x, y), radius=8, color=(0, 255, 255), thickness=3)
                    x1 = x
                    y1 = y
                if id == 8:
                    cv2.circle(image, (x, y), radius=8, color=(0, 0, 255), thickness=3)
                    x2 = x
                    y2 = y
                    distance = ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5
                    cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 5)

                    # Adjust the volume based on hand distance
                    if distance > 50:
                        current_volume = int(alpha * distance + (1 - alpha) * current_volume)
                        current_volume = min(100, current_volume)
                    else:
                        current_volume = int(alpha * distance + (1 - alpha) * current_volume)
                        current_volume = max(0, current_volume)

    # Show the current volume as a vertical bar on the webcam image
    volume_bar_height = int((current_volume / 100) * frame_height)
    cv2.rectangle(image, (frame_width - 50, frame_height), (frame_width - 30, frame_height - volume_bar_height),
                  (0, 255, 0), -1)

    # Display volume level text
    cv2.putText(image, f"Volume: {current_volume}", (frame_width - 100, frame_height - 30), cv2.FONT_HERSHEY_SIMPLEX,
                0.5, (255, 255, 255), 1, cv2.LINE_AA)

    # Show the frame in a window titled 'Webcam'
    cv2.imshow('Webcam', image)

    # Check for the 'Esc' key (ASCII code 27) to exit the loop
    if cv2.waitKey(1) == 27:
        break

# Release the webcam and close the window
webcam.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [8]:
pip install --upgrade pycaw


Note: you may need to restart the kernel to use updated packages.


In [9]:
import cv2
import mediapipe as mp
import pyautogui
import pycaw

x1 = y1 = x2 = y2 = 0

# Open the webcam (use 0 for the default webcam)
webcam = cv2.VideoCapture(0)

# To capture our hands
my_hands = mp.solutions.hands.Hands()
drawing_utils = mp.solutions.drawing_utils

# Set the initial volume level
current_volume = 10

# Initialize the pycaw library
volume_control = pycaw.Pycaw()

while True:
    # Read a frame from the webcam
    ret, image = webcam.read()
    frame_height, frame_width, _ = image.shape

    # We need to convert our image into RGB format
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the frame with Mediapipe Hands
    output = my_hands.process(rgb_image)

    # Collecting all hands from the variable output
    if output.multi_hand_landmarks:
        for hand_landmarks in output.multi_hand_landmarks:
            drawing_utils.draw_landmarks(image, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS)
            # For collecting the landmarks
            landmarks = hand_landmarks.landmark
            # Collecting id
            for id, landmark in enumerate(landmarks):
                x = int(landmark.x * frame_width)
                y = int(landmark.y * frame_height)
                if id == 4:
                    cv2.circle(image, (x, y), radius=8, color=(0, 255, 255), thickness=3)
                    x1 = x
                    y1 = y
                if id == 8:
                    cv2.circle(image, (x, y), radius=8, color=(0, 0, 255), thickness=3)
                    x2 = x
                    y2 = y
                    distance = ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5
                    cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 5)

                    # Adjust the volume based on hand distance
                    if distance > 50:
                        current_volume += 1
                        volume_control.set_volume(current_volume)
                    else:
                        current_volume -= 1
                        volume_control.set_volume(current_volume)

    # Show the current volume as a vertical bar on the webcam image
    volume_bar_height = int((current_volume / 100) * frame_height)
    cv2.rectangle(image, (frame_width - 50, frame_height), (frame_width - 30, frame_height - volume_bar_height),
                  (0, 255, 0), -1)
    
    # Display volume level text
    cv2.putText(image, f"Volume: {current_volume}", (frame_width - 100, frame_height - 30), cv2.FONT_HERSHEY_SIMPLEX,
                0.5, (255, 255, 255), 1, cv2.LINE_AA)

    # Show the frame in a window titled 'Webcam'
    cv2.imshow('Webcam', image)

    # Check for the '


AttributeError: module 'pycaw' has no attribute 'Pycaw'

In [ ]:
import cv2
import mediapipe as mp
from pycaw.pycaw import AudioUtilities, ISimpleAudioVolume

x1 = y1 = x2 = y2 = 0

# Open the webcam (use 0 for the default webcam)
webcam = cv2.VideoCapture(0)

# To capture our hands
my_hands = mp.solutions.hands.Hands()
drawing_utils = mp.solutions.drawing_utils

# Set the initial volume level
current_volume = 10
alpha = 0.2  # Smoothing factor for the exponential moving average

# Get the default audio session
sessions = AudioUtilities.GetAllSessions()
volume_control = None
for session in sessions:
    if session.Process and session.Process.name() == "YourProcessNameHere.exe":
        volume_control = session._ctl.QueryInterface(ISimpleAudioVolume)
        break

if not volume_control:
    raise Exception("Could not find the audio control interface.")

while True:
    # Read a frame from the webcam
    ret, image = webcam.read()
    frame_height, frame_width, _ = image.shape

    # ... Rest of the code remains the same ...

    # Adjust the volume based on hand distance
    if distance > 50:
        current_volume = int(alpha * distance + (1 - alpha) * current_volume)
        current_volume = min(100, current_volume)
    else:
        current_volume = int(alpha * distance + (1 - alpha) * current_volume)
        current_volume = max(0, current_volume)

    # Set the system's audio volume
    if volume_control:
        volume_control.SetMasterVolume(current_volume / 100, None)

    # ... Rest of the code remains the same ...

# Rest of the code remains the same ...


In [11]:
import cv2
import mediapipe as mp
from pycaw.pycaw import AudioUtilities, ISimpleAudioVolume

x1 = y1 = x2 = y2 = 0

# Open the webcam (use 0 for the default webcam)
webcam = cv2.VideoCapture(0)

# To capture our hands
my_hands = mp.solutions.hands.Hands()
drawing_utils = mp.solutions.drawing_utils

# Set the initial volume level
current_volume = 10
alpha = 0.2  # Smoothing factor for the exponential moving average

# Get the default audio session
sessions = AudioUtilities.GetAllSessions()
volume_control = None
for session in sessions:
    if session.Process and session.Process.name() == "YourProcessNameHere.exe":
        volume_control = session._ctl.QueryInterface(ISimpleAudioVolume)
        break

if not volume_control:
    raise Exception("Could not find the audio control interface.")

while True:
    # Read a frame from the webcam
    ret, image = webcam.read()
    frame_height, frame_width, _ = image.shape

    # We need to convert our image into RGB format
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the frame with Mediapipe Hands
    output = my_hands.process(rgb_image)

    # Collecting all hands from the variable output
    if output.multi_hand_landmarks:
        for hand_landmarks in output.multi_hand_landmarks:
            drawing_utils.draw_landmarks(image, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS)
            # For collecting the landmarks
            landmarks = hand_landmarks.landmark
            # Collecting id
            for id, landmark in enumerate(landmarks):
                x = int(landmark.x * frame_width)
                y = int(landmark.y * frame_height)
                if id == 4:
                    cv2.circle(image, (x, y), radius=8, color=(0, 255, 255), thickness=3)
                    x1 = x
                    y1 = y
                if id == 8:
                    cv2.circle(image, (x, y), radius=8, color=(0, 0, 255), thickness=3)
                    x2 = x
                    y2 = y
                    distance = ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5
                    cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 5)

                    # Adjust the volume based on hand distance
                    if distance > 50:
                        current_volume = int(alpha * distance + (1 - alpha) * current_volume)
                        current_volume = min(100, current_volume)
                    else:
                        current_volume = int(alpha * distance + (1 - alpha) * current_volume)
                        current_volume = max(0, current_volume)

    # Set the system's audio volume
    if volume_control:
        volume_control.SetMasterVolume(current_volume / 100, None)

    # Show the current volume as a vertical bar on the webcam image
    volume_bar_height = int((current_volume / 100) * frame_height)
    cv2.rectangle(image, (frame_width - 50, frame_height), (frame_width - 30, frame_height - volume_bar_height),
                  (0, 255, 0), -1)

    # Display volume level text
    cv2.putText(image, f"Volume: {current_volume}", (frame_width - 100, frame_height - 30), cv2.FONT_HERSHEY_SIMPLEX,
                0.5, (255, 255, 255), 1, cv2.LINE_AA)

    # Show the frame in a window titled 'Webcam'
    cv2.imshow('Webcam', image)

    # Check for the 'Esc' key (ASCII code 27) to exit the loop
    if cv2.waitKey(1) == 27:
        break

# Release the webcam and close the window
webcam.release()
cv2.destroyAllWindows()


ImportError: cannot import name 'HRESULT' from 'ctypes' (/Users/sooryaprabhu/anaconda3/envs/snowflakes/lib/python3.11/ctypes/__init__.py)

In [ ]:
#most working model for voice control using hand
import cv2
import mediapipe as mp #for hand detection
import pyautogui # to set the system volume on macOS.
import subprocess # for executing system commands

x1 = y1 = x2 = y2 = 0

# Open the webcam (use 0 for the default webcam)
webcam = cv2.VideoCapture(0)

# To capture our hands
my_hands = mp.solutions.hands.Hands()
#module contains functions that can be used to draw landmarks and connections on the detected hands in the webcam frames. 
#These functions are helpful for visualizing the results of the hand detection process.
drawing_utils = mp.solutions.drawing_utils #his line imports the drawing_utils module from mediapipe.solutions and assigns it to the variable drawing_utils

# Set the initial volume level
current_volume = 50
alpha = 0.2  # Smoothing factor for the exponential moving average

while True:
    # Read a frame from the webcam
    ret, image = webcam.read()
    frame_height, frame_width, _ = image.shape

    # We need to convert our image into RGB format
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the frame with Mediapipe Hands
    output = my_hands.process(rgb_image)

    # Collecting all hands from the variable output
    if output.multi_hand_landmarks:
        for hand_landmarks in output.multi_hand_landmarks:
            drawing_utils.draw_landmarks(image, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS)
            # For collecting the landmarks
            landmarks = hand_landmarks.landmark
            # Collecting id
            for id, landmark in enumerate(landmarks):
                x = int(landmark.x * frame_width)
                y = int(landmark.y * frame_height)
                if id == 4:
                    cv2.circle(image, (x, y), radius=8, color=(0, 255, 255), thickness=3)
                    x1 = x
                    y1 = y
                if id == 8:
                    cv2.circle(image, (x, y), radius=8, color=(0, 0, 255), thickness=3)
                    x2 = x
                    y2 = y
                    distance = ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5
                    cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 5)

                    # Adjust the volume based on hand distance
                    if distance > 50:
                        current_volume = int(alpha * distance + (1 - alpha) * current_volume)
                        current_volume = min(100, current_volume)
                    else:
                        current_volume = int(alpha * distance + (1 - alpha) * current_volume)
                        current_volume = max(0, current_volume)

    # Show the current volume as a vertical bar on the webcam image
    volume_bar_height = int((current_volume / 100) * frame_height)
    cv2.rectangle(image, (frame_width - 50, frame_height), (frame_width - 30, frame_height - volume_bar_height),
                  (0, 255, 0), -1)

    # Display volume level text
    cv2.putText(image, f"Volume: {current_volume}", (frame_width - 100, frame_height - 30), cv2.FONT_HERSHEY_SIMPLEX,
                0.5, (255, 255, 255), 1, cv2.LINE_AA)

    # Show the frame in a window titled 'Webcam'
    cv2.imshow('Webcam', image)

    # Set the volume on macOS using AppleScript
    volume_percent = int(current_volume)
    subprocess.run(['osascript', '-e', f'set volume output volume {volume_percent}'])

    # Check for the 'Esc' key (ASCII code 27) to exit the loop
    if cv2.waitKey(1) == 27:
        break

# Release the webcam and close the window
webcam.release()
cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
